In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interactive, FloatSlider, Checkbox, IntSlider, Button, VBox, fixed
from scipy.signal import savgol_filter

In [2]:
# Стартовые данные
x = np.linspace(0, 1, 1000)
initial_amplitude = 1.0
initial_frequency = 1.0
initial_phase = 0.0
initial_noise_mean = 0.0
initial_noise_covariance = 0.0
initial_show_noise = True
initial_window_size = 1

In [3]:
def f(t, amplitude, frequency, phase):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * t + phase)
    return harmonic

In [4]:
# Function to create noise
def create_noise(x, noise_mean, noise_covariance):
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(x))
    return noise

In [5]:
noise = create_noise(x, initial_noise_mean, initial_noise_covariance)

In [6]:
# Создаем слайдеры и чекбокс
amplitude_slider = FloatSlider(value=initial_amplitude, min=0, max=10.0, step=0.1, description='Amplitude')
frequency_slider = FloatSlider(value=initial_frequency, min=0, max=10.0, step=0.1, description='Frequency')
phase_slider = FloatSlider(value=initial_phase, min=0, max=100, step=0.1, description='Phase')
noise_mean_slider = FloatSlider(value=initial_noise_mean, min=0, max=10, step=0.1, description='Noise mean')
noise_covariance_slider = FloatSlider(value=initial_noise_covariance, min=0, max=2, step=0.05, description='Noise covariance')
show_noise_checkbox = Checkbox(value=True,description='Show noise')   
window_size_slider = IntSlider(value=initial_window_size,min=1,max=200, step=1, description='Window size')

In [7]:
signal = f(x, initial_amplitude, initial_frequency, initial_phase)

In [8]:
last_noise = create_noise(x, initial_noise_mean, initial_noise_covariance)
last_noise_mean = initial_noise_mean
last_noise_covariance = initial_noise_covariance

In [9]:
# Функция для обновления графика при изменении слайдеров
def create_signals(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, window_size):
    fig = go.Figure()
    fig2 = go.Figure()
    clean_signal = f(x, amplitude, frequency, phase)
    
    if show_noise:
        global last_noise, last_noise_mean, last_noise_covariance
         # Check if noise_mean or noise_covariance has changed
        if last_noise_mean != noise_mean or last_noise_covariance != noise_covariance:
            noise = create_noise(x, noise_mean, noise_covariance)
            last_noise = noise
            last_noise_mean = noise_mean
            last_noise_covariance = noise_covariance
        else:
            noise = last_noise
        
        noisy_signal = clean_signal + noise
        fig.add_trace(go.Scatter(x=x, y=noisy_signal, mode='lines', name='Noizy sinusoid', line=dict(width=1, color='red')))
        fig2.add_trace(go.Scatter(x=x, y=clean_signal, mode='lines', name='clean_signal', line=dict(width=1, color='blue')))
        fig2.add_trace(go.Scatter(x=x, y=moving_average_filter(noisy_signal, window_size), mode='lines', name='moving_avg', line=dict(width=1, color='red')))
        
    fig.add_trace(go.Scatter(x=x, y=clean_signal, mode='lines', name='Sinusoid', line=dict(width=2, color='blue')))
    fig.update_layout(title='Sinusoid Plot',
                      xaxis_title='X',
                      yaxis_title='Y',
                      showlegend=True)
    fig2.update_layout(title='Filtered sinusoid',
                      xaxis_title='X',
                      yaxis_title='Y',
                      showlegend=True)
    fig.show()
    fig2.show()

In [10]:
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise,window_size):
    noise = create_noise(x, noise_mean, noise_covariance)
    create_signals(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise,window_size)

In [11]:
reset_button = Button(description="Reset to Initial")
def reset_parameters(button):
    amplitude_slider.value = initial_amplitude
    frequency_slider.value = initial_frequency
    phase_slider.value = initial_phase
    noise_mean_slider.value = initial_noise_mean
    noise_covariance_slider.value = initial_noise_covariance
    show_noise_checkbox.value = initial_show_noise
reset_button.on_click(reset_parameters)

In [12]:
def moving_average_filter(data, window_size):
    kernel = np.ones(window_size) / window_size
    filtered_data = np.convolve(data, kernel, mode='same')
    return filtered_data

In [13]:
def savgol(data, window_size, order):
    half_size = window_size // 2

    if window_size % 2 == 0:
        raise ValueError("Window size must be odd.")

    if order >= window_size or order < 0:
        raise ValueError("Order must be less than window size and non-negative.")

    result = np.zeros_like(data)

    for i in range(half_size, len(data) - half_size):
        window = np.arange(-half_size, half_size + 1)
        coefficients = np.polyfit(window, data[i - half_size:i + half_size + 1], order)
        result[i] = np.polyval(coefficients, 0)

    return result


In [14]:
# Create the interactive plot
interactive_plot = interactive(update_plot,
                               amplitude=amplitude_slider,
                               frequency=frequency_slider,
                               phase=phase_slider,
                               noise_mean=noise_mean_slider,
                               noise_covariance=noise_covariance_slider,
                               show_noise=show_noise_checkbox,
                               window_size=window_size_slider)
display(reset_button, interactive_plot)

Button(description='Reset to Initial', style=ButtonStyle())

interactive(children=(FloatSlider(value=1.0, description='Amplitude', max=10.0), FloatSlider(value=1.0, descri…